## Implementacja metody UTAstar dyskretnej

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

# Funkcja do wyznaczania funkcji użyteczności dla wersji dyskretnej
def UTA_function_discrete(values, reference_points, lambda_param=1):
    """
    Aproksymuje funkcje użyteczności dla wersji dyskretnej.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_values = []
    
    # Iterujemy przez alternatywy
    for val in values:
        alternative_utility = []
        
        # Iterujemy przez każde kryterium
        for i in range(len(val)):
            ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
            # Interpolacja liniowa dla każdego kryterium
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_value = interp_func(val[i]) * lambda_param  # Użycie λ
            alternative_utility.append(utility_value)
        
        utility_values.append(alternative_utility)
    
    return np.array(utility_values)

# Funkcja do porównania alternatyw (dyskretna wersja)
def UTAstar_discrete(values, reference_points, lambda_param=1):
    """
    Metoda UTA Star dla wersji dyskretnej (wielokryterialnej) uwzględniająca preferencje decydenta.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_discrete(values, reference_points, lambda_param)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution_idx = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution_idx, utility_values

## Implementacja UTAstar ciągłej

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

# Funkcja do wyznaczania funkcji użyteczności (ciągła wersja z preferencjami)
def UTA_function_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Aproksymuje funkcje użyteczności w przestrzeni ciągłej dla wielu kryteriów.
    Uwzględnia preferencje decydenta przez współczynnik λ i punkty referencyjne.
    
    :param values: Lista wartości kryteriów dla alternatyw
    :param reference_points: Lista punktów referencyjnych (np. [najlepszy, najgorszy, neutralny])
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Funkcje użyteczności
    """
    utility_values = []
    for i in range(len(reference_points[0])):  # Iterujemy po kryteriach
        ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
        if use_polynomial:
            # Przy użyciu funkcji aproksymacji wielomianowej
            coef = np.polyfit(ref_values, [0, 1, 0], 2)  # Przykład: funkcja kwadratowa
            poly_func = np.poly1d(coef)
            utility_values.append(poly_func(values[:, i]) * lambda_param)  # Użycie λ
        else:
            # Interpolacja liniowa
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_values.append(interp_func(values[:, i]) * lambda_param)  # Użycie λ
    return np.array(utility_values).T  # Zwracamy funkcje użyteczności dla wszystkich alternatyw

# Funkcja do porównania alternatyw (ciągła wersja z uwzględnieniem preferencji)
def UTAstar_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Metoda UTA Star dla wersji ciągłej (wielokryterialnej) uwzględniająca preferencje decydenta.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_continuous(values, reference_points, lambda_param, use_polynomial)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution, utility_values
